In [2]:
import pandas as pd
wine = pd.read_csv("https://bit.ly//wine_csv_data")

data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

In [3]:
from sklearn.model_selection import train_test_split

#전체를 훈련세트와 테스트 세트로 나눈다
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

In [4]:
#훈련세트를 다시 훈련세트 80%와 검증세트 20%로 나눈다.
#교차검증 -> 훈련세트를 통해 머신을 훈련하고 검증세트로 모델을 평가하여 최적의 매개변수를 찾아내고 훈련세트 + 검증세트를 합쳐서 최적의 매개변수로 만들어진 모델을 학습한다. 이후 처음에 분리시켜둔 테스트 세트를 통해 최종적으로 모델을 평가한다.

sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size=0.2, random_state=42)


In [5]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=42)

dt.fit(sub_input, sub_target)
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))


0.9971133028626413
0.864423076923077


In [6]:
#교차검증
from sklearn.model_selection import cross_validate
import numpy as np

#cross_validate를 통해서 교차검증을 진행할 수 있다.(단 cross_validate는 훈련세트를 섞어 폴드를 나누지 않으므로 사용할 분할기를 지정해 주어야함)
#아래는 분할기를 사용하지 않고 그냥 진행
scores = cross_validate(dt, train_input, train_target)

#fit_time과 score_time은 각각 훈련과 검증에 걸린 시간을 의미하고 test_score은 검증세트로 평가한 모델의 성능을 의미한다. 교차검증의 최종점수는 test_score에 담긴 점수들을 평균 계산하여 구할 수 있다.
print(scores,'\n')

print(np.mean(scores['test_score']))

{'fit_time': array([0.01858044, 0.02471566, 0.04114842, 0.03373289, 0.02847052]), 'score_time': array([0.00191164, 0.00179291, 0.00242734, 0.01061463, 0.00172591]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])} 

0.855300214703487


In [7]:
#분할기를 사용하여 다시 진행하면 위와 같은 결과를 얻을 수 있다.(위에서는 train_test_split으로 미리 섞어두었기 때문)
#기본적으로 회귀 문제에서는 KFold를, 분류 모델에서는 StratiftedKFold를 사용한다
from sklearn.model_selection import StratifiedKFold
scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())

print(np.mean(scores['test_score']))

0.855300214703487


In [8]:
#n_splits로 겹의 개수를 정하고 shuffle을 True로 설정해서 데이터를 무작위로 섞은 뒤 겹을 나눈다.
splitter = StratifiedKFold(n_splits = 10, shuffle = True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv = splitter)

print(np.mean(scores['test_score']))

0.8574181117533719


In [9]:
#하이퍼파라미터 튜닝 -> 하이퍼파라미터는 모델이 학습할 수 없어 사람이 직접 지정해야하는 파라미터를 말한다.
#GridSearchCV 클래스를 통해 교차검증과 하이퍼파라미터 탐색을 동시에 진행할 수 있다.
from sklearn.model_selection import GridSearchCV

#탐색할 매개변수의 값의 리스트를 딕셔너리로 만든다.(매개변수 이름을 정확히 입력해야한다.)
params = {'min_impurity_decrease':[0.0001,0.0002,0.0003,0.0004,0.0005]}

#GridSearchCV 모델 생성(n_jobs를 통해 사용되는 cpu의 개수를 조절할 수 있다.)
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)

gs.fit(train_input, train_target)
print(gs.score(train_input, train_target))
print(gs.score(test_input, test_target))

0.9615162593804117
0.8653846153846154


In [10]:
#GridSearchCV 모델은 학습을 시키면 최적의 성능을 내는 파라미터로 자동으로 설정되어 있다. 혹은 best_esitmator_ 속성에 저장되어 있다.
print(gs.best_estimator_)

#결과가 위와 같음을 알 수 있다.
dt = gs.best_estimator_
print(dt.score(train_input, train_target))
print(dt.score(test_input, test_target))

DecisionTreeClassifier(min_impurity_decrease=0.0001, random_state=42)
0.9615162593804117
0.8653846153846154


In [11]:
#cv_results_ 속성의 'mean_test_score' 매개변수 별로 측정한 점수가 나와있다.
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [12]:
#파라미터 여러개 설정해보기
#2번째 값은 포함시키지 않음, range는 정수만 가능
#매개변수 이름을 정확히 입력하지 않으면 fit 또는 score를 사용할 때 에러가 날 수 있음.
params = {'min_impurity_decrease': np.arange(0.0001,0.001,0.0001),
          'max_depth':range(5,20,1),
          'min_samples_split':range(2,100,10)}

In [ ]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

print(gs.score(train_input, train_target))
print(gs.score(test_input, test_target))

0.892053107562055
0.8615384615384616


In [ ]:
#2가지 방법으로 계산된 최적의 매개변수 확인
dt = gs.best_estimator_
print(dt)

print(gs.best_params_)

DecisionTreeClassifier(max_depth=14, min_impurity_decrease=np.float64(0.0004),
                       min_samples_split=12, random_state=42)
{'max_depth': 14, 'min_impurity_decrease': np.float64(0.0004), 'min_samples_split': 12}


In [ ]:
#uniform과 randint로 랜덤 서치
from scipy.stats import uniform, randint

rgen = randint(0,10)
print(rgen.rvs(10),'\n')

uni = uniform(0,1)
print(uni.rvs(10))

[4 2 4 6 6 3 7 7 7 4] 

[0.13957962 0.13327043 0.10699024 0.32252382 0.79091806 0.52245108
 0.95596884 0.1264491  0.74535128 0.64169411]


In [ ]:
#랜덤 서치를 매개변수 설정에 사용할 수 있다.